In [2]:
# data cleaning
import numpy as np
import pandas as pd

# feature engineering
from sklearn.feature_selection import mutual_info_regression
from sklearn.cluster import KMeans

# model
from xgboost import XGBRegressor

# error score
from sklearn.model_selection import cross_val_score

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# load data
data = pd.read_csv(r'train.csv', index_col='Id')
data_test = pd.read_csv(r'test.csv', index_col='Id')

In [4]:
data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
data_test.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


### Missing Values - Train Data

In [6]:
# missing data
total_missing = data.isnull().sum().sort_values(ascending=False)
percent_missing = (data.isnull().sum() / data.isnull().count())*100
missing_data = pd.concat([total_missing, percent_missing], axis=1,
                         keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
PoolQC,1453,99.520548
MiscFeature,1406,96.301370
Alley,1369,93.767123
Fence,1179,80.753425
FireplaceQu,690,47.260274
LotFrontage,259,17.739726
GarageYrBlt,81,5.547945
GarageCond,81,5.547945
GarageType,81,5.547945
GarageFinish,81,5.547945


#### Droping data columns with 15% missing value

In [7]:
data = data.drop(['PoolQC', 'MiscFeature', 'Alley', 
                  'Fence', 'FireplaceQu', 'LotFrontage'], 
                 axis=1)
data.shape

(1460, 74)

We going to write a function To find missing values in each columns

In [8]:
# missing values - function
def missing_values(data, n):
    total_missing = data.isnull().sum().sort_values(ascending=False)
    percent_missing = (data.isnull().sum() / data.isnull().count())*100
    missing_data = pd.concat([total_missing, percent_missing], axis=1,
                            keys=['Total', 'Percent'])
    return missing_data.head(n)

In [9]:
# missing data
missing_values(data, 14)

,Total,Percent
GarageType,81,5.547945
GarageYrBlt,81,5.547945
GarageFinish,81,5.547945
GarageQual,81,5.547945
GarageCond,81,5.547945
BsmtFinType2,38,2.602740
BsmtExposure,38,2.602740
BsmtQual,37,2.534247
BsmtCond,37,2.534247
BsmtFinType1,37,2.534247


#### Column - GarageType
NA - No Garage

so we can replace NA with No Garage

In [10]:
data.GarageType.unique()

array(['Attchd', 'Detchd', 'BuiltIn', 'CarPort', nan, 'Basment', '2Types'],
      dtype=object)

In [11]:
data.GarageType = data.GarageType.fillna('No Garage')
missing_values(data, 13)

,Total,Percent
GarageYrBlt,81,5.547945
GarageFinish,81,5.547945
GarageQual,81,5.547945
GarageCond,81,5.547945
BsmtExposure,38,2.602740
BsmtFinType2,38,2.602740
BsmtFinType1,37,2.534247
BsmtQual,37,2.534247
BsmtCond,37,2.534247
MasVnrArea,8,0.547945


#### Column - GarageYrBlt

In [12]:
data.GarageYrBlt.unique()

array([2003., 1976., 2001., 1998., 2000., 1993., 2004., 1973., 1931.,
       1939., 1965., 2005., 1962., 2006., 1960., 1991., 1970., 1967.,
       1958., 1930., 2002., 1968., 2007., 2008., 1957., 1920., 1966.,
       1959., 1995., 1954., 1953.,   nan, 1983., 1977., 1997., 1985.,
       1963., 1981., 1964., 1999., 1935., 1990., 1945., 1987., 1989.,
       1915., 1956., 1948., 1974., 2009., 1950., 1961., 1921., 1900.,
       1979., 1951., 1969., 1936., 1975., 1971., 1923., 1984., 1926.,
       1955., 1986., 1988., 1916., 1932., 1972., 1918., 1980., 1924.,
       1996., 1940., 1949., 1994., 1910., 1978., 1982., 1992., 1925.,
       1941., 2010., 1927., 1947., 1937., 1942., 1938., 1952., 1928.,
       1922., 1934., 1906., 1914., 1946., 1908., 1929., 1933.])

In [13]:
# finding index of missing values
idx = data[data['GarageYrBlt'].isnull()].index.tolist()
data.iloc[idx].head(6)

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
41,20,RL,8658,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,...,0,0,0,0,0,12,2006,WD,Abnorml,160000
50,20,RL,7742,Pave,Reg,Lvl,AllPub,Inside,Gtl,Sawyer,...,0,0,0,0,0,1,2007,WD,Normal,127000
80,50,RM,10440,Pave,Reg,Lvl,AllPub,Corner,Gtl,OldTown,...,128,0,0,0,0,5,2009,WD,Normal,110000
90,20,RL,8070,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,...,0,0,0,0,0,8,2007,WD,Normal,123600
91,20,RL,7200,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,...,0,0,0,0,0,7,2006,WD,Normal,109900
101,20,RL,10603,Pave,IR1,Lvl,AllPub,Inside,Gtl,NWAmes,...,0,0,0,0,0,2,2010,WD,Normal,205000


Filling missing values in GarageYrBlt column

In [14]:
data.GarageYrBlt = data.GarageYrBlt.fillna(method='bfill',
                                           axis=0)
missing_values(data, 12)

,Total,Percent
GarageFinish,81,5.547945
GarageQual,81,5.547945
GarageCond,81,5.547945
BsmtFinType2,38,2.602740
BsmtExposure,38,2.602740
BsmtFinType1,37,2.534247
BsmtQual,37,2.534247
BsmtCond,37,2.534247
MasVnrType,8,0.547945
MasVnrArea,8,0.547945


#### Column - GarageFinish
NA - No Garage

so we can replace NA with No Garage

In [15]:
data.GarageFinish.unique()

array(['RFn', 'Unf', 'Fin', nan], dtype=object)

In [16]:
data.GarageFinish = data.GarageFinish.fillna('No Garage')
missing_values(data, 11)

,Total,Percent
GarageQual,81,5.547945
GarageCond,81,5.547945
BsmtFinType2,38,2.602740
BsmtExposure,38,2.602740
BsmtCond,37,2.534247
BsmtQual,37,2.534247
BsmtFinType1,37,2.534247
MasVnrArea,8,0.547945
MasVnrType,8,0.547945
Electrical,1,0.068493


#### Column - GarageQual
NA - No Garage

so we can replace NA with No Garage

In [17]:
data.GarageQual.unique()

array(['TA', 'Fa', 'Gd', nan, 'Ex', 'Po'], dtype=object)

In [18]:
data.GarageQual = data.GarageQual.fillna('No Garage')
missing_values(data, 10)

,Total,Percent
GarageCond,81,5.547945
BsmtExposure,38,2.602740
BsmtFinType2,38,2.602740
BsmtQual,37,2.534247
BsmtCond,37,2.534247
BsmtFinType1,37,2.534247
MasVnrType,8,0.547945
MasVnrArea,8,0.547945
Electrical,1,0.068493
Functional,0,0.000000


#### Column - GarageCond
NA - No Garage

so we can replace NA with No Garage

In [19]:
data.GarageCond = data.GarageCond.fillna('No Garage')
missing_values(data, 9)

,Total,Percent
BsmtFinType2,38,2.602740
BsmtExposure,38,2.602740
BsmtFinType1,37,2.534247
BsmtQual,37,2.534247
BsmtCond,37,2.534247
MasVnrType,8,0.547945
MasVnrArea,8,0.547945
Electrical,1,0.068493
Functional,0,0.000000


#### Column - BsmtFinType2
NA - No Basement

so we can replace NA with No Basement

In [20]:
data.BsmtFinType2.unique()

array(['Unf', 'BLQ', nan, 'ALQ', 'Rec', 'LwQ', 'GLQ'], dtype=object)

In [21]:
data.BsmtFinType2 = data.BsmtFinType2.fillna('No Basement')
missing_values(data, 8)

,Total,Percent
BsmtExposure,38,2.602740
BsmtFinType1,37,2.534247
BsmtQual,37,2.534247
BsmtCond,37,2.534247
MasVnrType,8,0.547945
MasVnrArea,8,0.547945
Electrical,1,0.068493
HalfBath,0,0.000000


#### Column - BsmtExposure
NA - No Basement

so we can replace NA with No Basement

In [22]:
data.BsmtExposure.unique()

array(['No', 'Gd', 'Mn', 'Av', nan], dtype=object)

In [23]:
data.BsmtExposure = data.BsmtExposure.fillna('No Basement')
missing_values(data, 7)

,Total,Percent
BsmtFinType1,37,2.534247
BsmtCond,37,2.534247
BsmtQual,37,2.534247
MasVnrType,8,0.547945
MasVnrArea,8,0.547945
Electrical,1,0.068493
BsmtHalfBath,0,0.000000


#### Column - BsmtFinType1
NA - No Basement

so we can replace NA with No Basement

In [24]:
data.BsmtFinType1.unique()

array(['GLQ', 'ALQ', 'Unf', 'Rec', 'BLQ', nan, 'LwQ'], dtype=object)

In [25]:
data.BsmtFinType1 = data.BsmtFinType1.fillna('No Basement')
missing_values(data, 6)

,Total,Percent
BsmtQual,37,2.534247
BsmtCond,37,2.534247
MasVnrType,8,0.547945
MasVnrArea,8,0.547945
Electrical,1,0.068493
GarageType,0,0.000000


#### Column - BsmtQual
NA - No Basement

so we can replace NA with No Basement

In [26]:
data.BsmtQual.unique()

array(['Gd', 'TA', 'Ex', nan, 'Fa'], dtype=object)

In [27]:
data.BsmtQual = data.BsmtQual.fillna('No Basement')
missing_values(data, 5)

,Total,Percent
BsmtCond,37,2.534247
MasVnrType,8,0.547945
MasVnrArea,8,0.547945
Electrical,1,0.068493
MSSubClass,0,0.000000


#### Column - BsmtCond
NA - No Basement

So we can replace NA with No Basement

In [29]:
data.BsmtCond.unique()

array(['TA', 'Gd', nan, 'Fa', 'Po'], dtype=object)

In [30]:
data.BsmtCond = data.BsmtCond.fillna('No Basement')
missing_values(data, 4)

,Total,Percent
MasVnrType,8,0.547945
MasVnrArea,8,0.547945
Electrical,1,0.068493
BedroomAbvGr,0,0.000000


#### Column - MasVnrType
Cinder Block **NOT** found in the data set

So  we assuming that missing values are Cinder Blockes